In [1]:
using ParallelTemperingMonteCarlo
# using Pkg;Pkg.add("BenchmarkTools")
using BenchmarkTools, DelimitedFiles
using Random,StaticArrays

# Equilibration

The purpose of this notebook is to write functions aimed at equilibrating the ptmc run. It must iterate over every trajectory for, by default 20\% of the total cycles run with _no_ sampling, other than testing the $\geq$ condition to update the min/max energies

In [2]:
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details

mc_cycles = 400000 #default 20% equilibration cycles on top

mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)


Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

In [3]:
ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=true)

Total number of moves per MC cycle: 13



equilibration done
equilibration done


MC loop done.
[0.0003275710166429321, 0.00034038231628987886, 0.0003541875217686626, 0.0003771192914525007, 0.0003871271199260189, 0.00041166221823725176, 0.00043725182080965515, 0.0004481145198137168, 0.0004892415372905008, 0.000515104682943868, 0.0005564389189273487, 0.0005866300755424616, 0.0006168592165216777, 0.0006944227240404881, 0.0007638579947522499, 0.0008900401243935854, 0.0009648086561921595, 0.0010102312231438865, 0.001162060369344718, 0.0015909789581521694, 0.0017710695798688687, 0.0022553539037819675, 0.0027215931181169776, 0.003094108733468708, 0.003327247352742674, 0.0031633947149170456, 0.003019056592225612, 0.002654620244466205, 0.0023918416792751244, 0.0021811717549868217, 0.0020685078870861556, 0.00194995364088873]
done


# Restart Equilibration 

In [4]:
results_test,mc_params_test,mc_states_test,step = restart_ptmc(pot)

UndefVarError: UndefVarError: NVT not defined

In [6]:
testfile = open("testing.txt","w+")
write(testfile,"$ensemble \n")
write(testfile,"$(atom_move_frequency(move_strat)) $(vol_move_frequency(move_strat)) $(rot_move_frequency(move_strat))  \n")
close(testfile)
testfile = open("testing.txt","r+")
te=readdlm(testfile)
close(testfile)
te[1]

ens = Meta.parse(te[1])
ensem = eval(ens)
ensem == ensemble
te[2,:]


3-element Vector{Any}:
 13
  0
  0

In [7]:
eval(Meta.parse(te[1]))

NVT(13)